In [1]:
import torch
import torch_mlir

from transformers import BertForMaskedLM


/home/developer/miniconda3/envs/torch-mlir/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.autograd.profiler as profiler

# Wrap the bert model to avoid multiple returns problem


class BertTinyWrapper(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained(
            "prajjwal1/bert-tiny", return_dict=False)

    # def forward(self, data):
    #     with profiler.record_function('fwd'):
    #         o= self.bert(data)[0]
    #     return o

    def forward(self, data):
        return self.bert(data)[0]


model = BertTinyWrapper()
model.eval()
data = torch.randint(30522, (2, 128))


Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
with torch.profiler.profile(
    on_trace_ready=torch.profiler.tensorboard_trace_handler('../output/log'),
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
    record_shapes=True,
    with_stack=True,
) as p:
    model.forward(data)
print(p.key_averages().table(
    sort_by="self_cpu_time_total", row_limit=-1))


---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::addmm        66.87%       8.500ms        78.57%       9.988ms     713.429us            14  
                aten::copy_        13.31%       1.692ms        13.31%       1.692ms      62.667us            27  
                  aten::bmm         3.34%     425.000us         3.34%     425.000us     106.250us             4  
                 aten::gelu         3.30%     419.000us         3.30%     419.000us     139.667us             3  
    aten::native_layer_norm         3.12%     396.000us         3.28%     417.000us      69.500us             6  
                  aten::add         1.53%     195.000us         1.53%     195.000us     

/home/developer/miniconda3/envs/torch-mlir/lib/python3.11/site-packages/torch/autograd/profiler.py:204: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")
STAGE:2023-04-15 02:24:40 6343:6343 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-04-15 02:24:40 6343:6343 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-04-15 02:24:40 6343:6343 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [4]:
%load_ext tensorboard
# Click this and select the right log directory
# if you have already started tensorboard, it may lock your port
# access it, open terminal and click in PORTS tab, than open the on vscode


In [5]:

# with torch.profiler.profile(
#     schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
#     on_trace_ready=torch.profiler.tensorboard_trace_handler('./log'),
#     record_shapes=True,
#     profile_memory=True,
#     with_stack=True,
# ) as p:
#     o = model.forward(data)

# print(p.profiler().table())
# print(o)
